In [2]:
l = ['z50', 'z150', 'z250', 'z400', 'z500', 'z700', 'z850', 'z1000', 'q50', 'q150', 'q250', 'q400', 'q500', 'q700', 'q850', 'q1000', 't50', 't150', 't250', 't400', 't500', 't700', 't850', 't1000', 'u50', 'u150', 'u250', 'u400', 'u500', 'u700', 'u850', 'u1000', 'v50', 'v150', 'v250', 'v400', 'v500', 'v700', 'v850', 'v1000', 'w50', 'w150', 'w250', 'w400', 'w500', 'w700', 'w850', 'w1000']

In [7]:
l.index("w850")

46

# Normalisation

In [1]:
import xarray as xr
import zarr
import numcodecs
import numpy as np
import glob
import torch
import era5_data_proc
import os

RAW_ERA5_PATH = "/vol/bitbucket/bet20/dataset/era5/global_full"

## Checking MEPS files

In [7]:
import neural_lam.utils

stats = neural_lam.utils.load_dataset_stats("meps_example")

for k, v in stats.items():
    print(k, v.shape)

data_mean torch.Size([17])
data_std torch.Size([17])
flux_mean torch.Size([])
flux_std torch.Size([])


In [8]:
stats

{'data_mean': tensor([ 9.9905e+04,  1.0110e+05, -2.1808e+05,  3.4544e+05,  7.1781e-01,
          7.1459e-01,  2.7186e+02,  2.7192e+02,  2.3915e+02,  2.6271e+02,
          2.4146e-01,  8.8596e-01, -2.5360e+00, -3.9970e+00,  6.2696e+00,
          8.5077e+02,  5.1629e+04]),
 'data_std': tensor([2.4345e+03, 7.6800e+02, 1.0905e+05, 5.0245e+05, 1.4583e-01, 1.6231e-01,
         4.8396e+00, 4.4669e+00, 5.4922e+00, 3.9706e+00, 4.3646e+00, 6.7067e+00,
         4.2867e+00, 5.2042e+00, 3.1096e+00, 5.9784e+02, 1.0582e+03]),
 'flux_mean': tensor(911702.5625),
 'flux_std': tensor(1037960.1250)}

## ERA5 Norm

In [52]:
nc_files = glob.glob(f'{RAW_ERA5_PATH}/*.nc')
nc_files.sort()
nc_files = nc_files[:12]
nc_files

['/vol/bitbucket/bet20/dataset/era5/global_full/2022_01.nc',
 '/vol/bitbucket/bet20/dataset/era5/global_full/2022_02.nc',
 '/vol/bitbucket/bet20/dataset/era5/global_full/2022_03.nc',
 '/vol/bitbucket/bet20/dataset/era5/global_full/2022_04.nc',
 '/vol/bitbucket/bet20/dataset/era5/global_full/2022_05.nc',
 '/vol/bitbucket/bet20/dataset/era5/global_full/2022_06.nc',
 '/vol/bitbucket/bet20/dataset/era5/global_full/2022_07.nc',
 '/vol/bitbucket/bet20/dataset/era5/global_full/2022_08.nc',
 '/vol/bitbucket/bet20/dataset/era5/global_full/2022_09.nc',
 '/vol/bitbucket/bet20/dataset/era5/global_full/2022_10.nc',
 '/vol/bitbucket/bet20/dataset/era5/global_full/2022_11.nc',
 '/vol/bitbucket/bet20/dataset/era5/global_full/2022_12.nc']

In [53]:
parameter_sums = np.array([0]) # (N_vars, N_levels)
var_sums = np.array([0]) # (N_vars, N_levels)
total_points = 0

In [54]:
i = 0

for i in range(len(nc_files[:3])):
    data = xr.open_dataset(nc_files[i])
    data = era5_data_proc.uk_subset(data)

    num_points = len(data['time']) * len(data["latitude"]) * len(data["longitude"])
    total_points = total_points + num_points

    sums = np.array([data[var].sum(['time', 'latitude', 'longitude']).values for var in data.data_vars])
    print(sums.shape)
    parameter_sums = parameter_sums + sums
    
    vals = data.to_array().values
    vals = np.transpose(vals, (0, 2, 1, 3, 4))
    print(vals.shape)
    var_sums = var_sums + np.sum(vals, axis=(2, 3, 4))

    print(nc_files[i])
    print(num_points)
    
np.isclose(parameter_sums, var_sums).all()

(6, 8)
(6, 8, 124, 65, 57)
/vol/bitbucket/bet20/dataset/era5/global_full/2022_01.nc
459420
(6, 8)
(6, 8, 112, 65, 57)
/vol/bitbucket/bet20/dataset/era5/global_full/2022_02.nc
414960
(6, 8)
(6, 8, 124, 65, 57)
/vol/bitbucket/bet20/dataset/era5/global_full/2022_03.nc
459420


True

## ERA5 Normalisation from Xarray

In [ ]:
var_sums = np.array([0]) # (N_vars, N_levels)
total_points = 0

for i in range(len(nc_files[:3])):
    data = xr.open_dataset(nc_files[i])
    data = era5_data_proc.uk_subset(data)

    num_points = len(data['time']) * len(data["latitude"]) * len(data["longitude"])
    total_points = total_points + num_points

    vals = data.to_array().values # (N_vars, N_times, N_levels, N_lats, N_lons)
    vals = np.transpose(vals, (0, 2, 1, 3, 4)) # (N_vars, N_levels, N_times, N_lats, N_lons)
    var_sums = var_sums + np.sum(vals, axis=(2, 3, 4)) # (N_vars, N_levels)
    
    print(nc_files[i])
    print(num_points)
    
var_means = var_sums / total_points

var_stds = np.array([0]) # (N_vars, N_levels)
for i in range(len(nc_files[:3])):
    data = xr.open_dataset(nc_files[i])
    data = era5_data_proc.uk_subset(data)

    vals = data.to_array().values # (N_vars, N_times, N_levels, N_lats, N_lons)
    vals = np.transpose(vals, (0, 2, 1, 3, 4)) # (N_vars, N_levels, N_times, N_lats, N_lons)
    var_squared_diff = (vals - var_means.reshape(*var_means.shape, 1, 1, 1)) ** 2
    var_stds = var_stds + np.sum(var_squared_diff, axis=(2, 3, 4)) # (N_vars, N_levels)
    
    print(nc_files[i])
    print(num_points)


/vol/bitbucket/bet20/dataset/era5/global_full/2022_01.nc
459420
/vol/bitbucket/bet20/dataset/era5/global_full/2022_02.nc
414960
/vol/bitbucket/bet20/dataset/era5/global_full/2022_03.nc
459420
/vol/bitbucket/bet20/dataset/era5/global_full/2022_01.nc
459420
/vol/bitbucket/bet20/dataset/era5/global_full/2022_02.nc
459420
/vol/bitbucket/bet20/dataset/era5/global_full/2022_03.nc
459420


In [74]:
var_means

array([[ 1.98566816e+05,  1.31345204e+05,  9.99637114e+04,
         6.94915794e+04,  5.39460403e+04,  3.31474312e+04,
         1.41598198e+04,  1.26508139e+03],
       [ 2.98321743e-06,  3.56682817e-06,  1.98416057e-05,
         1.88308410e-04,  4.45394453e-04,  1.11086685e-03,
         2.72969379e-03,  4.69097381e-03],
       [ 2.11742015e+02,  2.14685919e+02,  2.16439728e+02,
         2.36899708e+02,  2.48358562e+02,  2.62084934e+02,
         2.72406160e+02,  2.79824492e+02],
       [ 2.52679674e+01,  1.99496947e+01,  1.96528285e+01,
         1.57890576e+01,  1.36483263e+01,  1.05181391e+01,
         7.96240956e+00,  3.91867843e+00],
       [-2.01051010e+00, -2.21252477e+00, -2.24673299e+00,
        -3.94239711e-01,  3.00932753e-01,  8.27003147e-01,
         1.69441152e+00,  2.29910446e+00],
       [ 2.05483847e-03,  3.22094799e-03,  4.52772609e-04,
         8.94847508e-04, -2.17357494e-03, -6.33947440e-03,
        -5.22065071e-03, -1.16850669e-03]])

## ERA5 Normalisation with dataloader

In [76]:
import glob
import os
import torch
from tqdm import tqdm
from neural_lam.era5_dataset import ERA5UKDataset
from neural_lam import utils

dataset = "era5_uk_full"
batch_size = 2
n_workers = 4

In [77]:
static = utils.load_static_data(dataset)

In [79]:
static["data_std"]

tensor([2.5464e+03, 2.0264e+03, 2.2975e+03, 2.0187e+03, 1.7620e+03, 5.8213e+03,
        1.2267e+03, 1.1808e+03, 1.6962e-07, 1.1838e-06, 1.2919e-05, 1.5098e-04,
        3.6187e-04, 9.0703e-04, 1.3290e-03, 1.1877e-03, 6.1122e+00, 5.6558e+00,
        3.5887e+00, 4.8513e+00, 5.2299e+00, 5.2967e+00, 4.2371e+00, 2.8118e+00,
        1.4652e+01, 1.2807e+01, 1.9964e+01, 1.7005e+01, 1.4406e+01, 1.1302e+01,
        9.8326e+00, 7.3897e+00, 1.1996e+01, 1.2976e+01, 2.1520e+01, 1.8191e+01,
        1.5043e+01, 1.1692e+01, 9.7814e+00, 7.3720e+00, 3.0445e-02, 6.3717e-02,
        1.3932e-01, 2.6964e-01, 3.2182e-01, 3.4959e-01, 3.4065e-01, 1.1852e-01])

In [82]:
ds = ERA5UKDataset(
    dataset,
    split="train",
    standardize=False,
)  # Without standardization

In [83]:
loader = torch.utils.data.DataLoader(
    ds, batch_size, shuffle=False, num_workers=n_workers
)

In [73]:
means = []
squares = []
flux_means = []
flux_squares = []
i = 0
LIMIT = (31 * 4 + 28 * 4 + 31 * 4) // batch_size
for init_batch, target_batch, forcing_batch in tqdm(loader):
    if i == LIMIT:
        break
    i += 1
    batch = torch.cat(
        (init_batch, target_batch), dim=1
    )  # (N_batch, N_t, N_grid, d_features)
    means.append(torch.mean(batch, dim=(1, 2)))  # (N_batch, d_features,)
    squares.append(
        torch.mean(batch**2, dim=(1, 2))
    )  # (N_batch, d_features,)


mean = torch.mean(torch.cat(means, dim=0), dim=0)  # (d_features)

 25%|██▌       | 180/716 [00:10<00:32, 16.52it/s]


# Test ERA5 dataset

## Train Split

In [1]:
import glob
import os
import torch
from neural_lam.era5_dataset import ERA5UKDataset

dataset_name = "era5_uk_full"
batch_size = 2
n_workers = 4


In [5]:
train_set = ERA5UKDataset(
    dataset_name,
    pred_length=1,
    split="train",
    standardize=False,
)
train_loader = torch.utils.data.DataLoader(
    train_set,
    batch_size,
    shuffle=True,
    num_workers=n_workers,
)

print(len(train_loader))
dataiter = iter(train_loader)
init_states, target_states, forcing = next(dataiter)
init_states

729


tensor([[[[ 1.9736e+05,  1.3203e+05,  1.0197e+05,  ..., -2.6944e-01,
           -2.1301e-01, -1.2030e-02],
          [ 1.9744e+05,  1.3210e+05,  1.0204e+05,  ..., -2.9039e-01,
           -1.7324e-01, -3.4319e-03],
          [ 1.9754e+05,  1.3218e+05,  1.0212e+05,  ..., -2.5708e-01,
           -6.5231e-02,  1.8601e-02],
          ...,
          [ 2.0049e+05,  1.3320e+05,  1.0142e+05,  ...,  4.6545e-02,
            4.2540e-01, -9.6399e-02],
          [ 2.0052e+05,  1.3320e+05,  1.0142e+05,  ...,  1.3091e-01,
            2.2012e-01, -3.8899e-02],
          [ 2.0055e+05,  1.3319e+05,  1.0142e+05,  ...,  1.5402e-01,
           -2.2430e-01,  9.8133e-02]],

         [[ 1.9758e+05,  1.3228e+05,  1.0215e+05,  ..., -1.1360e-01,
            1.1694e-01, -1.1252e-01],
          [ 1.9765e+05,  1.3235e+05,  1.0221e+05,  ..., -9.8804e-03,
            1.9138e-02, -1.3778e-01],
          [ 1.9772e+05,  1.3242e+05,  1.0227e+05,  ...,  7.9325e-02,
           -6.4693e-02, -1.1628e-01],
          ...,
     

In [4]:
train_set = ERA5UKDataset(
    dataset_name,
    pred_length=1,
    split="train",
    standardize=True,
)
train_loader = torch.utils.data.DataLoader(
    train_set,
    batch_size,
    shuffle=True,
    num_workers=n_workers,
)

print(len(train_loader))
dataiter = iter(train_loader)
init_states, target_states, forcing = next(dataiter)
init_states

729


tensor([[[[ 2.9441,  2.0760,  1.3319,  ...,  0.7017,  0.3570, -0.1295],
          [ 2.9493,  2.1082,  1.3931,  ...,  0.7084,  0.3536, -0.0852],
          [ 2.9531,  2.1405,  1.4528,  ...,  0.6300,  0.3673, -0.0114],
          ...,
          [ 2.9980,  3.2260,  2.6165,  ...,  0.7734,  0.6669, -0.6510],
          [ 2.9916,  3.2405,  2.6165,  ...,  1.1254,  0.8740, -0.1344],
          [ 2.9865,  3.2550,  2.6151,  ...,  1.1254,  0.8997,  1.0021]],

         [[ 2.9018,  2.1098,  1.4884,  ..., -2.6577,  0.3793, -0.2131],
          [ 2.9095,  2.1502,  1.5538,  ..., -3.5968,  0.0506, -0.4788],
          [ 2.9159,  2.1921,  1.6164,  ..., -3.8821, -0.5536, -0.5477],
          ...,
          [ 2.9005,  3.1873,  2.5810,  ..., -0.1290,  0.9322, -0.0507],
          [ 2.8941,  3.1970,  2.5781,  ...,  0.0828,  1.2078,  0.2100],
          [ 2.8877,  3.2099,  2.5753,  ...,  0.1179,  1.2232,  1.0464]]],


        [[[ 3.1045,  1.8560,  0.5640,  ..., -0.2758, -0.7146, -0.1385],
          [ 3.1032,  1.8738,

In [88]:
path = "/vol/bitbucket/bet20/neural-lam/data/era5_uk_full/samples/train"
files = glob.glob(os.path.join(path, "*.npy"))
len(files)

1460

tensor([[[[ 2.0524e+05,  1.3322e+05,  9.9049e+04,  ...,  1.1326e-01,
           -1.4440e-02, -5.1097e-03],
          [ 2.0525e+05,  1.3323e+05,  9.9068e+04,  ...,  1.3484e-01,
           -5.9924e-02,  1.3047e-03],
          [ 2.0525e+05,  1.3325e+05,  9.9091e+04,  ...,  1.2143e-01,
           -1.8522e-02, -1.0278e-03],
          ...,
          [ 2.0605e+05,  1.3706e+05,  1.0462e+05,  ...,  3.1328e-01,
            1.0160e-01,  7.7191e-03],
          [ 2.0604e+05,  1.3711e+05,  1.0469e+05,  ...,  2.6721e-01,
            1.4300e-01,  4.8034e-03],
          [ 2.0603e+05,  1.3717e+05,  1.0475e+05,  ...,  1.7216e-01,
            8.1193e-02, -3.3100e-02]],

         [[ 2.0510e+05,  1.3317e+05,  9.9026e+04,  ..., -5.4675e-02,
           -5.9924e-02,  1.0052e-02],
          [ 2.0511e+05,  1.3318e+05,  9.9026e+04,  ...,  4.9704e-02,
           -4.8261e-02,  1.0052e-02],
          [ 2.0512e+05,  1.3320e+05,  9.9026e+04,  ...,  4.4456e-02,
           -7.9167e-02,  7.1359e-03],
          ...,
     

In [90]:
print(init_states.shape)
print(target_states.shape)
print(forcing.shape)

torch.Size([2, 2, 3705, 48])
torch.Size([2, 1, 3705, 48])
torch.Size([2, 1, 3705, 0])


In [5]:
val_set = ERA5UKDataset(
    dataset_name,
    pred_length=28,
    split="val",
)

val_loader = torch.utils.data.DataLoader(
    val_set,
    batch_size,
    shuffle=True,
    num_workers=n_workers,
)

valiter = iter(val_loader)
init_states, target_states, forcing = next(valiter)

print(len(val_set))
print(init_states.shape)
print(target_states.shape)
print(forcing.shape)

376
torch.Size([2, 2, 3705, 48])
torch.Size([2, 28, 3705, 48])
torch.Size([2, 28, 3705, 0])


## Train Val Split

In [8]:
import os
import glob
import datetime

In [23]:
# load time step data
sample_dir_path = "data/era5_uk_full/samples/train"
sample_files = glob.glob(f'{sample_dir_path}/*.npy')
sample_files.sort()
sample_files = [os.path.basename(f)[:-4] for f in sample_files]
sample_times = [datetime.datetime.strptime(f, '%Y%m%d%H%M%S') for f in sample_files]

In [24]:
sample_files[-1]

'20221231180000'

In [14]:
for i in range(1, len(sample_times)):
    delta = sample_times[i] - sample_times[i-1]
    if delta != datetime.timedelta(hours=6):
        print(f"Missing time step: {sample_times[i-1]} -> {sample_times[i]} ({delta})")

Missing time step: 2022-12-31 18:00:00 -> 2023-02-01 00:00:00 (31 days, 6:00:00)


In [53]:
sample_dir_path = "data/era5_uk_full/samples/val"
# directories = [name for name in os.listdir(sample_dir_path)]
# directories.sort()
# directories
val_months = ["01", "04", "07", "10"]

In [58]:
month_path = os.path.join(sample_dir_path, val_months[1])
print(month_path)
sample_files = glob.glob(f'{month_path}/*.npy')
sample_files.sort()
sample_files

data/era5_uk_full/samples/val/04


['data/era5_uk_full/samples/val/04/20230401000000.npy',
 'data/era5_uk_full/samples/val/04/20230401060000.npy',
 'data/era5_uk_full/samples/val/04/20230401120000.npy',
 'data/era5_uk_full/samples/val/04/20230401180000.npy',
 'data/era5_uk_full/samples/val/04/20230402000000.npy',
 'data/era5_uk_full/samples/val/04/20230402060000.npy',
 'data/era5_uk_full/samples/val/04/20230402120000.npy',
 'data/era5_uk_full/samples/val/04/20230402180000.npy',
 'data/era5_uk_full/samples/val/04/20230403000000.npy',
 'data/era5_uk_full/samples/val/04/20230403060000.npy',
 'data/era5_uk_full/samples/val/04/20230403120000.npy',
 'data/era5_uk_full/samples/val/04/20230403180000.npy',
 'data/era5_uk_full/samples/val/04/20230404000000.npy',
 'data/era5_uk_full/samples/val/04/20230404060000.npy',
 'data/era5_uk_full/samples/val/04/20230404120000.npy',
 'data/era5_uk_full/samples/val/04/20230404180000.npy',
 'data/era5_uk_full/samples/val/04/20230405000000.npy',
 'data/era5_uk_full/samples/val/04/2023040506000

In [60]:
import shutil
import glob
import os

# Specify the source and destination directories
src_dir = 'data/era5_uk_full/samples/train/'
dst_dir = 'data/era5_uk_full/samples/val/01'

os.makedirs(dst_dir, exist_ok=True)

# Use glob to get all .npy files that start with 2023 in the source directory
files = glob.glob(os.path.join(src_dir, '202301*.npy'))

# print(files)
# print(len(files))

# Move each file to the destination directory
for file in files:
    shutil.move(file, dst_dir)

In [42]:
31 * 4

124

In [75]:

pred_length = 28
sample_length = pred_length + 2
val_samples = []
sample_dir_path = "./data/era5_uk_full/samples/val"
month_samples = {}

for month in val_months:
    month_dir = os.path.join(sample_dir_path, month)
    data_files = glob.glob(os.path.join(month_dir, "*.npy"))
    data_files.sort()
    month_samples[month] = data_files
    n_samples = len(data_files) - sample_length + 1
    for i in range(n_samples):
        val_samples.append((month, i))
    

N = len(val_samples)
N

376

In [77]:
idx = 332
month, idx = val_samples[idx]
sample_names = month_samples[month]

sample_names[idx], idx

('./data/era5_uk_full/samples/val/10/20231013180000.npy', 51)

In [48]:
# verify number of time steps
val_files = glob.glob(f'{src_dir}/10/*.npy')
len(val_files)

124

In [35]:
val_files = glob.glob(f'{src_dir}/*.npy')
# val_files.sort()
# wanted = [v for v in val_files if "202301" in v or "202310" in v or "202304" in v or "202307" in v]
# wanted

[]

# Modulus GraphCast 

In [32]:
import torch

In [33]:
# lat, lon
input_res = (721, 1440)
latitudes = torch.linspace(-90, 90, steps=input_res[0])
latitudes

tensor([-90.0000, -89.7500, -89.5000, -89.2500, -89.0000, -88.7500, -88.5000,
        -88.2500, -88.0000, -87.7500, -87.5000, -87.2500, -87.0000, -86.7500,
        -86.5000, -86.2500, -86.0000, -85.7500, -85.5000, -85.2500, -85.0000,
        -84.7500, -84.5000, -84.2500, -84.0000, -83.7500, -83.5000, -83.2500,
        -83.0000, -82.7500, -82.5000, -82.2500, -82.0000, -81.7500, -81.5000,
        -81.2500, -81.0000, -80.7500, -80.5000, -80.2500, -80.0000, -79.7500,
        -79.5000, -79.2500, -79.0000, -78.7500, -78.5000, -78.2500, -78.0000,
        -77.7500, -77.5000, -77.2500, -77.0000, -76.7500, -76.5000, -76.2500,
        -76.0000, -75.7500, -75.5000, -75.2500, -75.0000, -74.7500, -74.5000,
        -74.2500, -74.0000, -73.7500, -73.5000, -73.2500, -73.0000, -72.7500,
        -72.5000, -72.2500, -72.0000, -71.7500, -71.5000, -71.2500, -71.0000,
        -70.7500, -70.5000, -70.2500, -70.0000, -69.7500, -69.5000, -69.2500,
        -69.0000, -68.7500, -68.5000, -68.2500, -68.0000, -67.75

In [34]:
longitudes = torch.linspace(-180, 180, steps=input_res[1] + 1)[1:]
longitudes

tensor([-179.7500, -179.5000, -179.2500,  ...,  179.5000,  179.7500,
         180.0000])

In [35]:
lat_lon_grid = torch.stack(
    torch.meshgrid(latitudes, longitudes, indexing="ij"), dim=-1
)

In [36]:
lat_lon_grid.shape

torch.Size([721, 1440, 2])

In [47]:
import os
import numpy as np

In [54]:
nwp_xy_path = os.path.join("./data/era5_uk_full/", "static", "nwp_xy.npy")
local_lat_lon_grid = np.load(nwp_xy_path) # (2, lon, lat) or (2, x, y)
local_lat_lon_grid = torch.from_numpy(local_lat_lon_grid).permute(2, 1, 0) # (lat, lon, 2)
print(local_lat_lon_grid.shape)
local_lat_lon_grid_flat = local_lat_lon_grid.permute(2, 0, 1).reshape(2, -1).permute(1, 0)
print(local_lat_lon_grid_flat.shape)

torch.Size([65, 57, 2])
torch.Size([3705, 2])


RuntimeError: view size is not compatible with input tensor's size and stride (at least one dimension spans across two contiguous subspaces). Use .reshape(...) instead.

# Sanity Check Graph

In [55]:
import numpy as np
import torch
import os

In [56]:
dir = "./data/era5_uk_full/static/nwp_xy.npy"
grid_xy = torch.tensor(np.load(dir))
_, lon, lat = grid_xy.shape

graph_name = "uk_graphcast"
graph_dir_path = os.path.join("graphs", graph_name)

In [57]:
print(lon, lat)
print("Local grid nodes", lon * lat)

X, Y = 1440, 721
fraction = (lon * lat) / (X * Y)
print("Global grid nodes", X * Y)
print("Local : Global", fraction)

57 65
Local grid nodes 3705
Global grid nodes 1038240
Local : Global 0.0035685390661118815


### Verify M2M Graph

In [61]:
N_MESH_NODES = 40962
N_MESH_EDGES = 327660 

mesh_features = torch.load(os.path.join(graph_dir_path, "mesh_features.pt"))
m2m_features = torch.load(os.path.join(graph_dir_path, "m2m_features.pt"))
m2m_edge_index = torch.load(os.path.join(graph_dir_path, "m2m_edge_index.pt"))

print(mesh_features[0].shape)
print(m2m_features[0].shape)
print(m2m_edge_index[0].shape)
print(m2m_edge_index[0].unique().shape)

print("Expected mesh nodes", N_MESH_NODES * fraction)
print("Expected mesh edges", N_MESH_EDGES * fraction)

torch.Size([355, 3])
torch.Size([2584, 4])
torch.Size([2, 2584])
torch.Size([355])
Expected mesh nodes 146.1744972260749
Expected mesh edges 1169.2675104022192


### Verify G2M Graph

In [63]:
N_G2M_EDGES = 1618746

g2m_features = torch.load(os.path.join(graph_dir_path, "g2m_features.pt"))
g2m_edge_index = torch.load(os.path.join(graph_dir_path, "g2m_edge_index.pt"))

print(g2m_features.shape)
print(g2m_edge_index.shape)
print(g2m_edge_index[0].unique().shape)
print(g2m_edge_index[1].unique().shape)
print(lat * lon)
print("Expected g2m edges", N_G2M_EDGES * fraction)

torch.Size([7719, 4])
torch.Size([2, 7719])
torch.Size([3705])
torch.Size([355])
3705
Expected g2m edges 5776.558339112344


### Verify M2G Graph

In [64]:
N_M2G_EDGES = 3114720

m2g_features = torch.load(os.path.join(graph_dir_path, "m2g_features.pt"))
m2g_edge_index = torch.load(os.path.join(graph_dir_path, "m2g_edge_index.pt"))

print(m2g_features.shape)
print(m2g_edge_index.shape)
print(m2g_edge_index[0].unique().shape)
print(m2g_edge_index[1].unique().shape)
print(lat * lon)
print("Expected m2g edges", N_M2G_EDGES * fraction)

torch.Size([11094, 4])
torch.Size([2, 11094])
torch.Size([355])
torch.Size([3705])
3705
Expected m2g edges 11115.0


### Verify against full graphcast

In [ ]:
num_grid_nodes = 
num_m2m_edges = 327660
num_m2g_edges = 3114720
  

```
g2m graph: Graph(num_nodes={'grid': 3705, 'mesh': 355},
      num_edges={('grid', 'g2m', 'mesh'): 7719},
      metagraph=[('grid', 'mesh', 'g2m')])
m2g graph: Graph(num_nodes={'grid': 3705, 'mesh': 355},
      num_edges={('mesh', 'm2g', 'grid'): 11094},
      metagraph=[('mesh', 'grid', 'm2g')])
mesh graph: Graph(num_nodes=355, num_edges=2584,
      ndata_schemes={'x': Scheme(shape=(3,), dtype=torch.float32)}
      edata_schemes={'x': Scheme(shape=(4,), dtype=torch.float32)})
```

# Parameter weighting

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
pressure_levels = np.array([1, 2, 3, 5, 7, 10, 20, 30, 50, 70, 100, 125, 150, 175, 200, 225, 250, 300, 350,
400, 450, 500, 550, 600, 650, 700, 750, 775, 800, 825, 850, 875, 900, 925, 950, 975, 1000])
# x_axis = np.linspace(0, )

weighting = pressure_levels / sum(pressure_levels)
fig, ax = plt.subplots()

ax.plot(pressure_levels, weighting, marker='o')
y_ticks = np.arange(0, 0.07, 0.005)
ax.set_yticks(y_ticks)

In [ ]:
subset_levels = [int(i) for i in ['50', '150', '250', '400', '500', '600', '850', '1000']]
subset_weighting = np.array([weighting[pressure_levels == i][0] for i in subset_levels])
subset_weighting

# Download Progress

In [ ]:
a = """2022_02.nc  2022_04.nc  2022_06.nc  2022_08.nc  2022_10.nc  2022_12.nc  2023_03.nc  2023_05.nc  2023_07.nc
2022_03.nc  2022_05.nc  2022_07.nc  2022_09.nc  2022_11.nc  2023_02.nc  2023_04.nc  2023_06.nc  2023_08.nc
"""

b = a.split()
b.sort()

In [ ]:
b

# PyTorch and NumPy

In [ ]:
import torch

In [ ]:
batch_size = 4
N_grid = 250
d_features = 42

prev_prev_state = torch.rand((batch_size, N_grid, d_features))
prev_state = torch.rand((batch_size, N_grid, d_features))
# sample_len = 5
target_states = [torch.rand((N_grid, d_features)), torch.rand((N_grid, d_features)), torch.rand((N_grid, d_features))]
target_states = torch.stack(target_states, dim=0)
target_states = target_states.unsqueeze(0).repeat(batch_size, 1, 1, 1)

forcings = torch.zeros(batch_size, target_states.shape[1], target_states.shape[2], 0) # (batch_size, sample_len-2, N_grid, d_forcing)
forcing = forcings[:, 2]

In [ ]:
print(prev_prev_state.shape)
print(prev_state.shape)
print(target_states.shape)
print(forcings.shape)
print(forcing.shape)

In [ ]:
grid_features = torch.cat(
    (
        prev_prev_state,
        prev_state,
        forcing
    ),
    dim=-1
)

In [ ]:
grid_features.shape

# ERA5 Dataset

In [13]:
import xarray as xr
import zarr
import numcodecs
import numpy as np
import glob
import torch
import era5_data_proc
import os

RAW_ERA5_PATH = "/vol/bitbucket/bet20/dataset/era5/global_full"

### Verify nwp_xy.npy and time step files

In [14]:
nc_files = glob.glob(f'{RAW_ERA5_PATH}/*.nc')
nc_files.sort()
print(nc_files[0])
data = xr.open_dataset(nc_files[0])
data = era5_data_proc.uk_subset(data)
time = data['time'].values[0]
print(time)
sample = data.sel(time=time)
sample

/vol/bitbucket/bet20/dataset/era5/global_full/2022_02.nc
2022-02-01T00:00:00.000000000


<xarray.Dataset> Size: 1MB
Dimensions:    (level: 8, latitude: 65, longitude: 57)
Coordinates:
  * longitude  (longitude) float32 228B 350.0 350.2 350.5 350.8 ... 3.5 3.75 4.0
  * latitude   (latitude) float32 260B 63.0 62.75 62.5 62.25 ... 47.5 47.25 47.0
  * level      (level) int32 32B 50 150 250 400 500 700 850 1000
    time       datetime64[ns] 8B 2022-02-01
Data variables:
    z          (level, latitude, longitude) float64 237kB 1.915e+05 ... 2.237...
    q          (level, latitude, longitude) float64 237kB 3.163e-06 ... 0.00278
    t          (level, latitude, longitude) float64 237kB 191.8 192.0 ... 275.5
    u          (level, latitude, longitude) float64 237kB 58.46 58.45 ... 1.674
    v          (level, latitude, longitude) float64 237kB -13.31 ... -1.532
    w          (level, latitude, longitude) float64 237kB 0.06596 ... -0.05892
Attributes:
    Conventions:  CF-1.6
    history:      2024-05-14 03:03:28 GMT by grib_to_netcdf-2.28.1: /opt/ecmw...

In [21]:
sample.data_vars

Data variables:
    z        (level, latitude, longitude) float64 237kB 1.915e+05 ... 2.237e+03
    q        (level, latitude, longitude) float64 237kB 3.163e-06 ... 0.00278
    t        (level, latitude, longitude) float64 237kB 191.8 192.0 ... 275.5
    u        (level, latitude, longitude) float64 237kB 58.46 58.45 ... 1.674
    v        (level, latitude, longitude) float64 237kB -13.31 -13.86 ... -1.532
    w        (level, latitude, longitude) float64 237kB 0.06596 ... -0.05892

In [22]:
list(sample.data_vars.keys())

['z', 'q', 't', 'u', 'v', 'w']

In [155]:
# load time step data
sample_dir_path = "data/era5_uk_full/samples/train"
sample_files = glob.glob(f'{sample_dir_path}/*.npy')
sample_files.sort()
print(sample_files[0])

time_step_data = np.load(sample_files[0])
print(time_step_data.shape)
time_step_data

data/era5_uk_full/samples/train/20220201000000.npy
(3705, 48)


array([[ 1.91469817e+05,  1.28869306e+05,  9.80167239e+04, ...,
         1.55718399e-01, -7.10621181e-02, -5.15267539e-03],
       [ 1.91662365e+05,  1.29052226e+05,  9.82734544e+04, ...,
         1.63957079e-01,  1.26255295e-02,  6.55492298e-03],
       [ 1.91874167e+05,  1.29244774e+05,  9.85366031e+04, ...,
         1.69616771e-02,  5.94559230e-02,  1.91297509e-02],
       ...,
       [ 1.98045325e+05,  1.32508460e+05,  1.01980000e+05, ...,
        -8.88403230e-02, -7.49646509e-02, -1.12689135e-01],
       [ 1.98077417e+05,  1.32521296e+05,  1.01957536e+05, ...,
         4.73147099e-02,  1.30568743e-01, -9.96806919e-02],
       [ 1.98099881e+05,  1.32543760e+05,  1.01935072e+05, ...,
        -4.67796918e-02,  3.08784407e-01, -5.89209050e-02]])

In [158]:
# load nwp_xy.npy
static_dir_path = "data/era5_uk_full/static"
grid_xy = torch.tensor(
    np.load(os.path.join(static_dir_path, "nwp_xy.npy"))
)  # (2, N_x, N_y)
print(grid_xy.shape)
grid_xy = grid_xy.reshape(2, -1).T # (N_x * N_y, 2)

torch.Size([2, 57, 65])


In [160]:
# choose random lon/lat point and verify alignment
test_point = 1204
test_xy = grid_xy[test_point] % 360
print(test_xy)
time_step_data[test_point]

tensor([354.5000,  54.5000])


array([ 1.96437551e+05,  1.33695838e+05,  1.03359926e+05,  7.20259750e+04,
        5.59193484e+04,  3.01885391e+04,  1.47237386e+04,  1.67862264e+03,
        3.16304429e-06,  4.29940966e-06,  3.76327939e-05,  3.73618156e-04,
        4.91421366e-04,  3.61225947e-03,  6.34824849e-03,  5.99711159e-03,
        1.96218200e+02,  1.96649870e+02,  2.18782948e+02,  2.45168774e+02,
        2.58076905e+02,  2.72899571e+02,  2.78619198e+02,  2.81463024e+02,
        4.32877382e+01,  3.54867076e+01,  3.45491500e+01,  3.30573304e+01,
        3.36474402e+01,  2.72334433e+01,  1.81308622e+01,  1.38153398e+01,
       -2.51317416e+01, -2.97051160e+01, -3.31565113e+01, -3.68174162e+01,
       -3.95630949e+01, -2.40097625e+01, -1.82730583e+01, -3.28485048e+00,
        8.89417263e-02, -9.92470771e-02, -2.42317246e-02, -1.76864118e-01,
        2.34658984e-02, -2.94351016e-02, -2.16300361e-02,  4.03312423e-01])

In [161]:
sample.sel(longitude=test_xy[0], latitude=test_xy[1])

<xarray.Dataset> Size: 432B
Dimensions:    (level: 8)
Coordinates:
    longitude  float32 4B 354.5
    latitude   float32 4B 54.5
  * level      (level) int32 32B 50 150 250 400 500 700 850 1000
    time       datetime64[ns] 8B 2022-02-01
Data variables:
    z          (level) float64 64B 1.964e+05 1.337e+05 ... 1.472e+04 1.679e+03
    q          (level) float64 64B 3.163e-06 4.299e-06 ... 0.006348 0.005997
    t          (level) float64 64B 196.2 196.6 218.8 245.2 ... 272.9 278.6 281.5
    u          (level) float64 64B 43.29 35.49 34.55 33.06 ... 27.23 18.13 13.82
    v          (level) float64 64B -25.13 -29.71 -33.16 ... -24.01 -18.27 -3.285
    w          (level) float64 64B 0.08894 -0.09925 -0.02423 ... -0.02163 0.4033
Attributes:
    Conventions:  CF-1.6
    history:      2024-05-14 03:03:28 GMT by grib_to_netcdf-2.28.1: /opt/ecmw...

## ERA5 Dataset Names

In [38]:
sample.level.values

array([  50,  150,  250,  400,  500,  700,  850, 1000], dtype=int32)

In [43]:
sample["units"]

KeyError: "No variable named 'units'. Variables on the dataset include ['z', 'q', 't', 'u', 'v', 'w', 'longitude', 'latitude', 'level', 'time']"

In [46]:
from neural_lam.constants import ERA5UKConstants

PARAM_SYMBOLS = list(sample.data_vars.keys())
LEVELS = list(sample.level.values)
PARAM_NAMES = [
    p + str(l)
    for p in PARAM_SYMBOLS
    for l in LEVELS
]

PARAM_UNITS = [
    sample[p].attrs["units"]
    for p in PARAM_SYMBOLS
    for _ in LEVELS
]

print(PARAM_NAMES)
print(PARAM_UNITS)

['z50', 'z150', 'z250', 'z400', 'z500', 'z700', 'z850', 'z1000', 'q50', 'q150', 'q250', 'q400', 'q500', 'q700', 'q850', 'q1000', 't50', 't150', 't250', 't400', 't500', 't700', 't850', 't1000', 'u50', 'u150', 'u250', 'u400', 'u500', 'u700', 'u850', 'u1000', 'v50', 'v150', 'v250', 'v400', 'v500', 'v700', 'v850', 'v1000', 'w50', 'w150', 'w250', 'w400', 'w500', 'w700', 'w850', 'w1000']
['m**2 s**-2', 'm**2 s**-2', 'm**2 s**-2', 'm**2 s**-2', 'm**2 s**-2', 'm**2 s**-2', 'm**2 s**-2', 'm**2 s**-2', 'kg kg**-1', 'kg kg**-1', 'kg kg**-1', 'kg kg**-1', 'kg kg**-1', 'kg kg**-1', 'kg kg**-1', 'kg kg**-1', 'K', 'K', 'K', 'K', 'K', 'K', 'K', 'K', 'm s**-1', 'm s**-1', 'm s**-1', 'm s**-1', 'm s**-1', 'm s**-1', 'm s**-1', 'm s**-1', 'm s**-1', 'm s**-1', 'm s**-1', 'm s**-1', 'm s**-1', 'm s**-1', 'm s**-1', 'm s**-1', 'Pa s**-1', 'Pa s**-1', 'Pa s**-1', 'Pa s**-1', 'Pa s**-1', 'Pa s**-1', 'Pa s**-1', 'Pa s**-1']


## Grid Features

In [180]:
grid_features_path = "/vol/bitbucket/bet20/neural-lam/data/era5_uk_full/static/grid_features.pt"
grid_features = torch.load(grid_features_path)
grid_features.shape

torch.Size([3705, 5])

## Verify static variables

In [165]:
dataset_path = "/vol/bitbucket/bet20/dataset/era5/global_full/static_variables.nc"
data = xr.open_dataset(dataset_path)
data

<xarray.Dataset> Size: 266MB
Dimensions:    (longitude: 1440, latitude: 721, time: 16)
Coordinates:
  * longitude  (longitude) float32 6kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
  * latitude   (latitude) float32 3kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * time       (time) datetime64[ns] 128B 2022-03-11T09:00:00 ... 2023-10-28T...
Data variables:
    z          (time, latitude, longitude) float64 133MB ...
    lsm        (time, latitude, longitude) float64 133MB ...
Attributes:
    Conventions:  CF-1.6
    history:      2024-05-15 15:41:55 GMT by grib_to_netcdf-2.28.1: /opt/ecmw...

In [168]:
# surface geopotential should be time invariant but we find that it is not.
# values changed sometime in 2022 October
# 99.7% of the values are the same however
sample1 = data.sel(time=data['time'].values[3])["z"].values
sample2 = data.sel(time=data['time'].values[4])["z"].values
np.isclose(sample1, sample2).sum() / sample1.size

0.9972260748959778

## Junk

In [ ]:
dataset_path = "data/era5_uk_reduced/samples/train"
nc_files = glob.glob(f'{constants.DATASET_PATH}/global/*.nc')
print(nc_files)
data = xr.open_dataset(nc_files[0])

In [ ]:
data

In [ ]:
from era5_data_proc import uk_subset
uk_data = uk_subset(data)
uk_data

In [ ]:
uk_data['longitude'] = xr.where(uk_data['longitude'] > 180, uk_data['longitude'] - 360, uk_data['longitude'])

In [ ]:
uk_data

In [ ]:
uk_data.isel(time=0).to_array().values.shape

In [ ]:
latitude = uk_data.latitude.values
latitude

In [ ]:
longitudes = uk_data.longitude.values
longitudes = np.where(longitudes > 180, longitudes - 360, longitudes)
longitudes

In [ ]:
latitudes = uk_data.latitude.values
longitudes = uk_data.longitude.values
# print(latitudes.shape, longitudes.shape)

t_lat = torch.from_numpy(latitudes)
t_lon = torch.from_numpy(longitudes)

lat_lon_grid = torch.stack(
    torch.meshgrid(t_lat, t_lon, indexing="ij"), dim=-1
).permute(2, 1, 0)

lat_lon_grid.shape # (2, lon, lat) or (2, x, y)

grid_array = lat_lon_grid.numpy()
grid_array.shape
# np.save("nwp_xy.npy", grid_array)

In [ ]:
np.load("data/era5_uk_reduced/static/nwp_xy.npy")

In [ ]:
processed_data_path="data/era5_uk_reduced/samples/train"
sample_name="20220101000000"
sample_path=f"{processed_data_path}/{sample_name}.npy"

sample=np.load(sample_path)

In [ ]:
import torch

In [ ]:
# lat, lon
input_res = (721, 1440)
latitudes = torch.linspace(-90, 90, steps=input_res[0])
latitudes

In [ ]:
longitudes = torch.linspace(-180, 180, steps=input_res[1] + 1)[1:]
longitudes

In [ ]:
lat_lon_grid = torch.stack(
    torch.meshgrid(latitudes, longitudes, indexing="ij"), dim=-1
)

In [ ]:
lat_lon_grid.shape

In [ ]:
lat_lon_grid

In [ ]:
torch.from_numpy(np.load("data/era5_global/static/nwp_xy.npy")).permute(2, 1, 0)

In [ ]:
print(lat_lon_grid.shape)

In [ ]:
tmp = lat_lon_grid.permute(2, 0, 1)
tmp.shape

In [ ]:
tmp[1][0].shape

In [ ]:
tmp.view(2, -1).shape

In [ ]:
import json
import numpy as np

icospheres_path = "data/era5_global/icospheres.json"
with open(icospheres_path, "r") as f:
    loaded_dict = json.load(f)
    icospheres = {
        key: (np.array(value) if isinstance(value, list) else value)
        for key, value in loaded_dict.items()
    }

In [ ]:
for k in icospheres.keys():
    print(k)

In [ ]:
icospheres["order_0_face_centroid"]

In [ ]:
from graphcast_utils import latlon2xyz
from sklearn.neighbors import NearestNeighbors

input_res = (721, 1440)
latitudes = torch.linspace(-90, 90, steps=input_res[0])
longitudes = torch.linspace(-180, 180, steps=input_res[1] + 1)[1:]
lat_lon_grid = torch.stack(
    torch.meshgrid(latitudes, longitudes, indexing="ij"), dim=-1
)
lat_lon_grid_flat = lat_lon_grid.permute(2, 0, 1).view(2, -1).permute(1, 0)
lat_lon_grid_flat.shape

In [ ]:
cartesian_grid = latlon2xyz(lat_lon_grid_flat) # (lat*lon, 3)
n_nbrs = 4
neighbors = NearestNeighbors(n_neighbors=n_nbrs).fit(
    icospheres["order_" + str(6) + "_vertices"] # (40962, 3)
)

In [ ]:
distances, indices = neighbors.kneighbors(cartesian_grid) # (lat*lon, n_nbrs)

In [ ]:
lat_lon_grid_flat

In [ ]:
cartesian_grid

In [ ]:
src, dst = [], []
for i in range(len(cartesian_grid)):
    for j in range(n_nbrs):
        if distances[i][j] <= 0.6 * edge_len:
            src.append(i)
            dst.append(indices[i][j])

In [ ]:
np.load("data/era5_uk_reduced/samples/train/20220101000000.npy").shape # (6, 7, 65, 57) (levels, variables, lat, lon)
np.load("data/_era5_uk_reduced/samples/train/20220101000000.npy").shape # (6, 7, 721, 1440)

In [ ]:
uk_bbox = {
    "lat_max": 63,
    "lat_min": 47,
    "lon_max": 4,
    "lon_min": -10,
}

torch.linspace(uk_bbox["lat_min"], uk_bbox["lat_max"], steps=65) # (65,)

In [ ]:
torch.linspace(uk_bbox["lon_min"], uk_bbox["lon_max"], steps=57) # (65,)

In [ ]:
import torch

# Assuming larger is your larger tensor and smaller is your smaller tensor
larger = torch.tensor([[1, 2], [3, 4], [5, 6], [7, 8]])
smaller = torch.tensor([[1, 2], [7, 8]])

# Expand dimensions for broadcasting
larger_exp = larger.unsqueeze(1)
smaller_exp = smaller.unsqueeze(0)

# Compare all pairs of rows
matches = torch.all(larger_exp == smaller_exp, dim=-1)

# Get indices of matching rows
indices = torch.where(matches)

In [ ]:
indices

In [ ]:
import torch

# Example tensors
larger_tensor = torch.tensor([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0], [7.0, 8.0]])
smaller_tensor = torch.tensor([[5.0, 6.0], [3.0, 4.0], [1.0, 2.0]])

def find_subset_indices(larger, smaller):
    """
    finds the indices of the smaller tensor in the larger tensor
    """
    indices = []
    for i in range(smaller.size(0)):
        index = torch.where(torch.all(larger == smaller[i], dim=1))[0]
        if index.size(0) == 0:
            raise ValueError("The smaller tensor is not a subset of the larger tensor")
        indices.append(index.item())
    return indices

indices = find_subset_indices(larger_tensor, smaller_tensor)
print(indices)

In [ ]:
import torch
import numpy as np

# Assuming tensor1 and tensor2 are your tensors
tensor1 = torch.tensor([[1, 2], [3, 4], [5, 6], [7, 8]])
tensor2 = torch.tensor([[1, 2], [7, 8], [9, 10]])

# Convert tensors to numpy arrays
array1 = tensor1.numpy()
array2 = tensor2.numpy()

# Find common rows
common_rows = np.intersect1d(array1, array2, axis=0)

# Convert back to tensor
common_rows_tensor = torch.from_numpy(common_rows)

In [ ]:
a = list(range(10))
b = [3, 2 , 1]

a[b]

In [ ]:
import neural_lam.constants as constants
w_dict = {
    "2": 1.0,
    "0": 0.1,
    "65": 0.065,
    "1000": 0.1,
    "850": 0.05,
    "500": 0.03,
}
w_list = np.array(
    [w_dict[par.split("_")[-2]] for par in constants.PARAM_NAMES]
)

w_list

# MEPS Dataset

In [ ]:
import numpy as np
import neural_lam.constants as constants
# Create parameter weights based on height
# based on fig A.1 in graph cast paper
w_dict = {
    "2": 1.0,
    "0": 0.1,
    "65": 0.065,
    "1000": 0.1,
    "850": 0.05,
    "500": 0.03,
}
w_list = np.array(
    [w_dict[par.split("_")[-2]] for par in constants.PARAM_NAMES]
)

In [ ]:
for w in zip(w_list, constants.PARAM_NAMES):
    print(w)

# Test the thing

In [ ]:
from neural_lam.constants import MEPSConstants

MEPSConstants.GRID_SHAPE

# Icospheres

In [ ]:
import numpy as np
import json

icospheres_path = "data/era5_uk/icospheres.json"

with open(icospheres_path, "r") as f:
    loaded_dict = json.load(f)
    icospheres = {
        key: (np.array(value) if isinstance(value, list) else value)
        for key, value in loaded_dict.items()
    }
    print(f"Opened pre-computed graph at {icospheres_path}.")

In [ ]:
for i in icospheres.keys():
    print(i)

In [ ]:
len([key for key in icospheres.keys() if "faces" in key]) - 2

In [ ]:
1440 * 720

In [ ]:
import torch

a = torch.tensor([1, 2, 3, 4, 5])
b = torch.tensor([6, 7, 8, 9, 10])

torch.stack((a, b))

# DGL

In [ ]:
from typing import List

import dgl
from dgl import DGLGraph
import torch
import numpy as np
 

def create_heterograph(
    src: List, dst: List, labels: str, dtype: torch.dtype = torch.int32
) -> DGLGraph:
    """Creates a heterogeneous DGL graph from an adj matrix in COO format.

    Parameters
    ----------
    src : List
        List of source nodes
    dst : List
        List of destination nodes
    labels : str
        Label of the edge type
    dtype : torch.dtype, optional
        Graph index data type, by default torch.int32

    Returns
    -------
    DGLGraph
        The dgl Graph.
    """
    graph = dgl.heterograph({labels: ("coo", (src, dst))}, idtype=dtype)
    return graph

src = [np.array([0, 1, 2]), np.array([3, 4, 5])]
dst = [np.array([1, 2, 3]), np.array([4, 5, 6])]

graph = create_heterograph(src, dst, ("mesh", "m2g", "grid"), dtype=torch.int32)


In [ ]:
graph

# Ridiculous

In [ ]:
"""
           5663073       gpu submit_j    bet20 PD       0:00      1 (Priority)
           5663071       gpu submit_j    bet20 PD       0:00      1 (Resources)
     5662324_[0-4]       gpu    F2mIF  qwang33 PD       0:00      1 (Priority)
     5662323_[1-5]       gpu    F2mIF  qwang33 PD       0:00      1 (Priority)
           5660755       gpu 24layers   cc2040 PD       0:00      2 (Priority)
           5660861       gpu    r-c16 yiweiche PD       0:00      1 (Priority)
           5660862       gpu    r-c32 yiweiche PD       0:00      1 (Priority)
           5660900       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5660898       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5660899       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5660897       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5660893       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5660894       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5660895       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5660896       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5660889       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5660890       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5660891       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5660892       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5660888       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5660887       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5660886       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5662524       gpu sr327mod gpw2-ec2 PD       0:00      4 (Priority)
           5662502       gpu    uw_G1   aa2201 PD       0:00      1 (Priority)
           5662706       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662707       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662704       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662705       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662702       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662703       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662701       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662699       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662700       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662697       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662698       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662696       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662694       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662695       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662692       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662693       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662691       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662689       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662690       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662687       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662688       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662686       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662684       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662685       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662682       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662683       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662681       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662679       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662680       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662677       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662678       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662675       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662676       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662671       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662672       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662673       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662674       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662669       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662670       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662665       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662666       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662667       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662668       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662663       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662664       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662659       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662660       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662661       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662662       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662657       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662658       gpu P23Hetde   cc2040 PD       0:00      1 (Priority)
           5662656       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662653       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662654       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662655       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662651       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662652       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662648       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662649       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662650       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662646       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662647       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662643       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662644       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662645       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662642       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662641       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662638       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662639       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662640       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662635       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662636       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662637       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662633       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662634       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662631       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662632       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662629       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662630       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662627       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662628       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662626       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662625       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662624       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662623       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662622       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662621       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662620       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662619       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662618       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662617       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662616       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662615       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662614       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662613       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662612       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662611       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662610       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662609       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5662608       gpu 3c353_id   cc2040 PD       0:00      1 (Priority)
           5663082       gpu uw_G1_ra   aa2201 PD       0:00      1 (Priority)
           5663081       gpu bio512_2   aa2201 PD       0:00      1 (Priority)
           5663080       gpu   bio512   aa2201 PD       0:00      1 (Priority)
           5663078       gpu ast512_2   aa2201 PD       0:00      1 (Priority)
           5663076       gpu   ast512   aa2201 PD       0:00      1 (Priority)
           5662472       gpu   MUMAX3      cpy PD       0:00      1 (Priority)
           5662195       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5662191       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5662192       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5662193       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5662194       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5662190       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5662186       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5662187       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5662188       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5662189       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5662182       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5662183       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5662184       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5662185       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5662201       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5662200       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5662199       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5662198       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5662197       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5662196       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5662473       gpu   MUMAX3      cpy PD       0:00      1 (Dependency)
           5662598       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5662597       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5662596       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5662595       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5662594       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5662593       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5662592       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5662591       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5662590       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5662589       gpu     traj xr223_ci PD       0:00      1 (Priority)
           5631081       gpu HemKtest   julian  R 13-03:43:08      1 r2i6n0
           5662497       gpu run_XCLI xingran0  R    4:51:28      4 r2i4n[1-4]
           5660733       gpu     mace juraskov  R   11:50:40      1 r2i5n7
           5660732       gpu     mace juraskov  R   11:59:47      1 r2i7n1
           5660731       gpu     mace juraskov  R   12:05:27      1 r2i5n7
         5660551_5       gpu    F2mIF  qwang33  R   13:54:11      1 r2i6n3
         5660551_4       gpu    F2mIF  qwang33  R   14:21:30      1 r2i4n0
           5660715       gpu     mace juraskov  R   12:37:19      1 r2i4n5
           5660716       gpu     mace juraskov  R   12:37:19      1 r2i4n5
           5660717       gpu     mace juraskov  R   12:37:19      1 r2i4n5
           5660718       gpu     mace juraskov  R   12:37:19      1 r2i4n5
           5660719       gpu     mace juraskov  R   12:37:19      1 r2i4n8
           5660720       gpu     mace juraskov  R   12:37:19      1 r2i4n8
           5660721       gpu     mace juraskov  R   12:37:19      1 r2i4n8
           5660722       gpu     mace juraskov  R   12:37:19      1 r2i4n8
           5660723       gpu     mace juraskov  R   12:37:19      1 r2i6n1
           5660724       gpu     mace juraskov  R   12:37:19      1 r2i6n1
           5660725       gpu     mace juraskov  R   12:37:19      1 r2i6n1
           5660726       gpu     mace juraskov  R   12:37:19      1 r2i6n1
           5660727       gpu     mace juraskov  R   12:37:19      1 r2i6n2
           5660728       gpu     mace juraskov  R   12:37:19      1 r2i6n2
           5660729       gpu     mace juraskov  R   12:37:19      1 r2i6n2
           5660730       gpu     mace juraskov  R   12:37:19      1 r2i6n2
           5660711       gpu     mace juraskov  R   13:54:11      1 r2i7n3
           5660712       gpu     mace juraskov  R   13:54:11      1 r2i7n3
           5660713       gpu     mace juraskov  R   13:54:11      1 r2i7n3
           5660714       gpu     mace juraskov  R   13:54:11      1 r2i7n3
         5660551_3       gpu    F2mIF  qwang33  R   15:29:46      1 r2i7n0
         5660551_1       gpu    F2mIF  qwang33  R   16:04:11      1 r2i7n2
         5660551_2       gpu    F2mIF  qwang33  R   16:04:11      1 r2i7n4
           5660710       gpu     mace juraskov  R   14:24:38      1 r2i6n0
           5660709       gpu     mace juraskov  R   14:50:53      1 r2i5n6
           5660708       gpu     mace juraskov  R   16:15:23      1 r2i6n0
           5660707       gpu     mace juraskov  R   16:16:24      1 r2i6n0
           5662126       gpu   myvenv zhenyaza  R   11:03:18      1 r2i4n6
           5662127       gpu   myvenv zhenyaza  R   11:03:18      1 r2i5n8
           5660706       gpu     mace juraskov  R   17:55:58      1 r2i7n6
           5660705       gpu     mace juraskov  R   18:04:32      1 r2i7n6
           5660704       gpu     mace juraskov  R   20:16:54      1 r2i6n4
           5660703       gpu     mace juraskov  R   20:18:55      1 r2i6n4
           5660702       gpu     mace juraskov  R   20:26:30      1 r2i5n6
           5660701       gpu     mace juraskov  R   20:33:04      1 r2i5n7
           5660699       gpu     mace juraskov  R   21:40:48      1 r2i7n6
           5660700       gpu     mace juraskov  R   21:40:48      1 r2i7n6
           5660698       gpu     mace juraskov  R   23:17:14      1 r2i4n7
           5656524       gpu     mace juraskov  R 3-21:13:59      1 r2i5n3
           5660697       gpu     mace juraskov  R 1-00:47:30      1 r2i5n6
           5660696       gpu     mace juraskov  R 1-00:50:01      1 r2i5n6
           5660695       gpu     mace juraskov  R 1-01:24:47      1 r2i4n7
           5660694       gpu     mace juraskov  R 1-02:37:16      1 r2i6n4
           5660693       gpu     mace juraskov  R 1-02:37:17      1 r2i6n4
         5662323_0       gpu    F2mIF  qwang33  R    7:29:11      1 r2i5n4
           5660753       gpu sr327mod gpw2-ec2  R    7:51:43      4 r2i6n[6-8],r2i7n5
           5660692       gpu     mace juraskov  R 1-03:30:36      1 r2i4n7
           5660686       gpu     mace juraskov  R 1-03:31:06      1 r2i4n7
           5660687       gpu     mace juraskov  R 1-03:31:06      1 r2i7n1
           5660688       gpu     mace juraskov  R 1-03:31:06      1 r2i7n1
           5660689       gpu     mace juraskov  R 1-03:31:06      1 r2i5n1
           5660690       gpu     mace juraskov  R 1-03:31:06      1 r2i5n1
           5660691       gpu     mace juraskov  R 1-03:31:06      1 r2i5n7
           5660885       gpu     traj xr223_ci  R    5:20:49      1 r2i7n7
           5660884       gpu     traj xr223_ci  R    5:21:19      1 r2i7n7
           5660883       gpu     traj xr223_ci  R    8:02:01      1 r2i7n7
           5660882       gpu     traj xr223_ci  R    8:03:31      1 r2i7n7
           5660881       gpu     traj xr223_ci  R   11:17:41      1 r2i7n1
           5657937       gpu       pd yiweiche  R 3-07:08:49      1 r2i5n1

"""